In [1]:
# 讀檔
import csv
with open('movies.csv', encoding = 'utf8') as csvfile:
    # 解析內容
    rows = list(csv.reader(csvfile))

In [2]:
#分配測試資料與訓練資料
test_movie = []
train_movie = []
for movie in rows[1:]:
    if movie[2] != 'NA':
        if len(test_movie) < 500:
            test_movie.append(movie)
        else:
            train_movie.append(movie)

#只取第一個分類
for movie in test_movie:
    temp = movie[2].split(",")
    movie[2] = temp[0]

for movie in train_movie:
    temp = movie[2].split(",")
    movie[2] = temp[0]

In [3]:
#有哪些分類
categories = []

for movie in train_movie:
    temp = movie[2].split(",")
    categories.append(temp[0])

categories = set(categories)
categories = list(categories)
print(categories)

['音樂/歌舞', '戲劇', '劇情', '恐怖', '冒險', '懸疑/驚悚', '動畫', '動作', '歷史/傳記', '愛情', '犯罪', '溫馨/家庭', '喜劇', '勵志', '奇幻', '影展', '紀錄片', '科幻', '武俠', '戰爭']


In [4]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import math
from sklearn.preprocessing import normalize

In [5]:
#資料分詞
vectorizer = TfidfVectorizer(sublinear_tf=False, stop_words=None, token_pattern="(?u)\\b\\w+\\b", smooth_idf=True, norm='l2')
data = []
for i in train_movie:
    data.append(i[3][-100:])
for i in test_movie:
    data.append(i[3][-100:])
tfidf = vectorizer.fit_transform(data)
tfidf = tfidf.toarray()
#df_tfidf = pd.DataFrame(tfidf.toarray(),columns=vectorizer.get_feature_names(), index=range(len(train_data))).transpose()

#df_tfidf
#print("TFIDF")

In [6]:
df_tfidf = pd.DataFrame(tfidf,columns=vectorizer.get_feature_names(), index=range(len(data)))
df_tfidf

,0,00,000個家庭出租社會住宅,000公里的,000殺害另一個目標莎拉,001,002,003,005,005年,...,龐德將不惜賠上個人一切,龐德的忠誠度再度面臨質疑考驗,李承鉉,理解,０,１,２,２人同行優惠價４００元,２月更是有4場總計10部話題短片席捲而來,ｋ羅琳
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#訓練資料 分類決定y值
y_train = []
for movie in train_movie:
    for i in range(len(categories)):
        if movie[2] == categories[i]:
            y_train.append(i)

In [8]:
#訓練資料 x值為tfidf
x_train = list(tfidf[:-500])

In [9]:
#資料分詞
'''
vectorizer = TfidfVectorizer(sublinear_tf=False, stop_words=None, token_pattern="(?u)\\b\\w+\\b", smooth_idf=True, norm='l2')
test_data = []
for i in test_movie:
    test_data.append(i[3][-50:-1])
tfidf = vectorizer.fit_transform(test_data)
tfidf = tfidf.toarray()
'''

'\nvectorizer = TfidfVectorizer(sublinear_tf=False, stop_words=None, token_pattern="(?u)\\b\\w+\\b", smooth_idf=True, norm=\'l2\')\ntest_data = []\nfor i in test_movie:\n    test_data.append(i[3][-50:-1])\ntfidf = vectorizer.fit_transform(test_data)\ntfidf = tfidf.toarray()\n'

In [10]:
#測試資料 x值為tfidf值
x_test = list(tfidf[-500:])

In [14]:
from sklearn.neighbors import KNeighborsClassifier

#knn k值=5開始測試
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(x_train,y_train)
pred = knn.predict(x_test)

In [15]:
#預測結果
y_test = []

for i in pred:
    y_test.append(categories[i])

In [16]:
#準確率
correct = 0
for i in range(500):
    if y_test[i] == test_movie[i][2]:
        correct = correct + 1

accuracy = correct / 500
print(accuracy)

0.348
